In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = 'data/doto/'

In [4]:
!ls {PATH}

hero_data.csv test1.csv     test9.csv     train1.csv    train9.csv


In [5]:
df_raw = pd.read_csv(f'{PATH}train9.csv', low_memory=False)

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [7]:
train_cats(df_raw)

In [8]:
display_all(df_raw.tail())

,user_id,hero_id,id,num_games,num_wins,kda_ratio
18841,2992,8,2992_8,97,59,4153.992395
18842,2992,34,2992_34,93,40,3571.428572
18843,2992,70,2992_70,89,53,4212.180747
18844,2992,76,2992_76,290,211,5471.003135
18845,2992,114,2992_114,89,58,4396.396396


In [9]:
df_raw = df_raw.drop('id',axis=1)

In [10]:
display_all(df_raw.tail())

,user_id,hero_id,num_games,num_wins,kda_ratio
18841,2992,8,97,59,4153.992395
18842,2992,34,93,40,3571.428572
18843,2992,70,89,53,4212.180747
18844,2992,76,290,211,5471.003135
18845,2992,114,89,58,4396.396396


In [11]:
df_raw['kda']=df_raw['kda_ratio']/df_raw['num_games']

In [12]:
display_all(df_raw.head())

,user_id,hero_id,num_games,num_wins,kda_ratio,kda
0,1,74,304,185,3743.699732,12.314802
1,1,65,255,151,4009.021513,15.721653
2,1,39,203,135,5094.621514,25.096658
3,1,106,158,106,4619.212963,29.235525
4,1,2,157,94,2871.627907,18.290624


In [13]:
user_ids = df_raw.user_id.unique()

In [14]:
user_ids

array([   1,    2,    5, ..., 2990, 2991, 2992])

In [15]:
df1 = df_raw.groupby('user_id').agg(lambda x: x.tolist())

In [16]:
df1.head()

,hero_id,num_games,num_wins,kda_ratio,kda
user_id,,,,,
1,"[74, 65, 39, 106, 2, 46, 51, 17, 69]","[304, 255, 203, 158, 157, 154, 128, 118, 116]","[185, 151, 135, 106, 94, 103, 78, 75, 65]","[3743.6997318999997, 4009.0215128, 5094.621513...","[12.314801749671052, 15.72165299137255, 25.096..."
2,"[74, 8, 11, 52, 73, 70, 39, 18, 76]","[657, 493, 487, 386, 241, 218, 205, 195, 172]","[362, 256, 229, 240, 123, 134, 106, 113, 85]","[3061.0596341, 2712.1680049, 2664.801343, 3007...","[4.659147083866058, 5.501354979513185, 5.47187..."
5,"[74, 46, 1, 93, 8, 17, 11, 109, 10]","[276, 251, 240, 237, 224, 212, 196, 176, 167]","[160, 162, 145, 137, 116, 122, 100, 105, 106]","[3899.4082841, 3785.1307189000004, 5032.214765...","[14.12829088442029, 15.08020206733068, 20.9675..."
6,"[74, 53, 13, 9, 19, 17, 80, 51, 25]","[295, 293, 247, 204, 202, 197, 186, 185, 185]","[166, 152, 128, 118, 121, 114, 104, 118, 107]","[3802.3019634999996, 3383.6784409999996, 4087....","[12.889159198305084, 11.548390583617746, 16.54..."
7,"[62, 91, 22, 67, 17, 106, 27, 75, 54]","[1512, 1107, 1058, 474, 424, 381, 300, 296, 293]","[855, 643, 568, 249, 200, 196, 167, 165, 140]","[3181.3036334000003, 2126.5707105, 3716.117468...","[2.1040367945767198, 1.9210214186991872, 3.512..."


In [17]:
count = 0
list_kda=[]
for x in range(len(df1)):
    list_kda.append((sum(df1.iloc[x]['kda'])/9))
#     print(list_kda/9)
    count+=1
count

2094

In [18]:
# list_kda

In [19]:
df1['kda_avg'] = list_kda

In [20]:
df1.head()

,hero_id,num_games,num_wins,kda_ratio,kda,kda_avg
user_id,,,,,,
1,"[74, 65, 39, 106, 2, 46, 51, 17, 69]","[304, 255, 203, 158, 157, 154, 128, 118, 116]","[185, 151, 135, 106, 94, 103, 78, 75, 65]","[3743.6997318999997, 4009.0215128, 5094.621513...","[12.314801749671052, 15.72165299137255, 25.096...",23.735499
2,"[74, 8, 11, 52, 73, 70, 39, 18, 76]","[657, 493, 487, 386, 241, 218, 205, 195, 172]","[362, 256, 229, 240, 123, 134, 106, 113, 85]","[3061.0596341, 2712.1680049, 2664.801343, 3007...","[4.659147083866058, 5.501354979513185, 5.47187...",10.385079
5,"[74, 46, 1, 93, 8, 17, 11, 109, 10]","[276, 251, 240, 237, 224, 212, 196, 176, 167]","[160, 162, 145, 137, 116, 122, 100, 105, 106]","[3899.4082841, 3785.1307189000004, 5032.214765...","[14.12829088442029, 15.08020206733068, 20.9675...",19.712627
6,"[74, 53, 13, 9, 19, 17, 80, 51, 25]","[295, 293, 247, 204, 202, 197, 186, 185, 185]","[166, 152, 128, 118, 121, 114, 104, 118, 107]","[3802.3019634999996, 3383.6784409999996, 4087....","[12.889159198305084, 11.548390583617746, 16.54...",18.194261
7,"[62, 91, 22, 67, 17, 106, 27, 75, 54]","[1512, 1107, 1058, 474, 424, 381, 300, 296, 293]","[855, 643, 568, 249, 200, 196, 167, 165, 140]","[3181.3036334000003, 2126.5707105, 3716.117468...","[2.1040367945767198, 1.9210214186991872, 3.512...",6.416719


In [21]:
train_raw = pd.read_csv(f'{PATH}train1.csv', low_memory=False)

In [22]:
display_all(train_raw.head())

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,1,41,1_41,201,125,3731.707317
1,2,17,2_17,427,234,2922.058381
2,5,106,5_106,240,130,5116.682739
3,6,65,6_65,206,122,4131.553861
4,7,69,7_69,697,382,2474.469306


In [23]:
train_cats(df_raw)

In [24]:
train_raw['kda']=list_kda

In [25]:
len(train_raw)

2094

In [26]:
train_raw['pred']=train_raw['kda']*train_raw['num_games']

In [27]:
y_actual = train_raw['kda_ratio']

In [28]:
y_predicted = train_raw['pred']

In [34]:
train_raw.head()

,user_id,hero_id,id,num_games,num_wins,kda_ratio,kda,pred
0,1,41,1_41,201,125,3731.707317,23.735499,4770.835378
1,2,17,2_17,427,234,2922.058381,10.385079,4434.428660
2,5,106,5_106,240,130,5116.682739,19.712627,4731.030378
3,6,65,6_65,206,122,4131.553861,18.194261,3748.017731
4,7,69,7_69,697,382,2474.469306,6.416719,4472.453035


In [29]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual, y_predicted))

In [30]:
rms

3484.873196226025

In [ ]:
# display_all(train_raw)

In [ ]:
df_raw = pd.read_csv(f'{PATH}test9.csv', low_memory=False)

In [ ]:
df_raw['kda']=df_raw['kda_ratio']/df_raw['num_games']

In [ ]:
df_raw = df_raw.drop('id',axis=1)

In [ ]:
# display_all(df_raw)

In [ ]:
len(df_raw)

In [ ]:
df1 = df_raw.groupby('user_id').agg(lambda x: x.tolist())

In [ ]:
len(df1)

In [ ]:
df1.head()

In [ ]:
count = 0
list_kda=[]
for x in range(len(df1)):
    list_kda.append((sum(df1.iloc[x]['kda'])/9))
#     print(list_kda/9)
    count+=1
count

In [ ]:
df1['kda_avg'] = list_kda

In [ ]:
df1.head()

In [ ]:
train_raw = pd.read_csv(f'{PATH}test1.csv', low_memory=False)

In [ ]:
train_raw.head()

In [ ]:
len(train_raw)

In [ ]:
train_raw['kda']=list_kda

In [ ]:
train_raw.head()

In [ ]:
train_raw['kda_ratio']=train_raw['kda']*train_raw['num_games']

In [ ]:
train_raw.head()

In [ ]:
ids = train_raw['id']

In [ ]:
preds= train_raw['kda_ratio']

In [ ]:
subm = np.stack([ids,preds], axis=1)


In [ ]:
submission_file_name = 'doto_kda_pred.csv'
np.savetxt(submission_file_name, subm, fmt='%s,%0.5f', header='id,kda_ratio', comments='')

In [ ]:
from IPython.display import FileLink
FileLink(submission_file_name)
